### Feature Engineering

In [ ]:
from ImgPrep import file_names, read_img, reshape_array, split_img, remove_background, fix_edges, flatten_arrays, glcmm
from ImgCluster import cluster_pixels, cluster_pieces, silhouette_analysis, optimalK
from Funcs4Testing import display_cluster_imgs, display_img, img_scatter, plot_silhouette_analysis, plot_gap_stats
from sklearn.manifold import Isomap
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.decomposition import PCA
from networkx.convert import to_networkx_graph as ToGraph
from collections import defaultdict
import cdlib
import joblib
import numpy as np
import pandas as pd
import re

#### SET GLOBAL VARIABLES HERE
# -------------------------

puzzle_folder = 'puzzle_2'
load_from_pickle = True # set to True to load feature set from pickle file, otherwise set to False and generate feature set

# -------------------------
###########################

# list of files to read in
scan_directory = 'puzzle_scans/' + puzzle_folder
files = file_names(scan_directory)
pictures = [] # picture arrays
bgremoved = [] # pic arrays with scanner background removed
avg_rgb_pieces = [] # avg rgp of the entire puzzle piece
pic_arrays = [] # flattened pic arrays
pic_arrays_filtered = [] # flattened pic arrays w/ [0,0,0] pixels filtered out
features = []

for i in range(1, len(files) + 1):
    
    # store picture as array
    pic = read_img(files, i)

    # split each image into 20 pieces
    pieces = split_img(pic, 5, 4)
    pictures += pieces
    assert all([piece.shape for piece in pieces]), 'All arrays are not the same size'

shape = pieces[0].shape

# generate features for data points

if load_from_pickle is True:
    filename = 'pickle_files/' + puzzle_folder + '/feature_set.pkl'
    feature_set = joblib.load(filename)
else:
    for indx,img in enumerate(pictures):
        # remove background
        pic_bgremoved = remove_background(img)
        pic_bgremoved = fix_edges(pic_bgremoved, indx, shape)
        assert pic_bgremoved.shape == img.shape, \
            'Picture with background removed is not the same size as original picture'
        bgremoved.append(pic_bgremoved)

        # flatten picture array
        flat_pic = reshape_array(pic_bgremoved, starting_dim = len(pic_bgremoved.shape))
        assert flat_pic.shape[0] == img.shape[0] * img.shape[1], \
            'Flattened picture does not have the same number of pixels as original picture'
        pic_arrays.append(flat_pic)

        # find average rgb of entire piece
        keep_indices = np.where(np.sum(flat_pic, axis = 1) > 0)[0]
        flat_pic_filtered = flat_pic[keep_indices]
        pic_arrays_filtered.append(flat_pic_filtered)
        avg_rgb_pieces.append(np.mean(flat_pic_filtered, axis = 0))

    assert len(bgremoved) == len(pictures), \
        'List of pictures with background removed is not the same size as original list of pictures'
    assert len(pic_arrays_filtered) == len(pictures), \
        'List of flattened picture arrays is not the same size as original list of pictures'
    assert len(avg_rgb_pieces) == len(pictures), \
        'Quantity of average rgb values % quantity of puzzle pieces' % ('exceeds' if len(avg_rgb_pieces) > len(pictures) else 'is less than')

    # Find RGB values of cluster centers for each puzzle piece
    clustered_pixels = cluster_pixels(pixels = pic_arrays_filtered, n_clusters = 3)

    # flatten cluster centers to use in features
    centers_flattened = flatten_arrays(clustered_pixels)

    # find texture features   
    txtr_features = np.empty((len(bgremoved),24))
    for indx,img in enumerate(bgremoved):
        txtr_features[indx] = glcmm(img)

    # concatenate all features for clustering pieces
    feature_set =  np.concatenate((avg_rgb_pieces, centers_flattened, txtr_features), axis = 1)

### Clustering

In [ ]:
# calculate gap-statistic to determine optimal k for clustering
k, gapdf = optimalK(feature_set, maxClusters = 10)
print(f'Optimal k is: {k}')

plot_gap_stats(gapdf, k)

In [ ]:
# perform Isomap and PCA on final feature set
km_iso_models = defaultdict(dict)
km_pca_models = defaultdict(dict)

for i in range(2, 15):
    # perform KMeans clustering on isomap reduction
    iso_name = 'isoKM_' + str(i)
    iso = Isomap(n_components = i).fit_transform(feature_set)
    km_iso_models[iso_name]['reduced_features'] = iso
    km_iso_models[iso_name]['model'] = KMeans(n_clusters = k).fit(iso)
    
    # perform KMeans clustering on pca reduction
    pca_name = 'pcaKM_' + str(i)
    pca = PCA(n_components = i).fit_transform(feature_set)
    km_pca_models[pca_name]['reduced_features'] = pca
    km_pca_models[pca_name]['model'] = KMeans(n_clusters = k).fit(pca)

# perform k-means clustering on all features
km = KMeans(n_clusters = k).fit(feature_set)

# Spectral clustering on all features
sc = SpectralClustering(n_clusters = k, assign_labels = "discretize", random_state = 5).fit(feature_set)

### Goodness of Fit
[Evaluation metrics provided by 'cdlib' package](https://cdlib.readthedocs.io/en/latest/reference/classes/node_clustering.html#methods)
<br>
[Additional cdlib documentation](https://cdlib.readthedocs.io/_/downloads/en/stable/pdf/)


In [ ]:
# compare clusters to adjacency graph
adjacency_path = 'AdjacencyMatrices/' + puzzle_folder.replace('_','').replace('p','P') + '_AdjacencyMatrix.csv'
adj = np.loadtxt(adjacency_path, delimiter = ',', dtype=int)
assert adj.shape[0] == adj.shape[1], 'Adjacency matrix must be symmetrical'
assert adj.shape[0] == len(pictures), 'Adjacency matrix should have one row and one column for every puzzle piece'

# Convert the adjacency matrix into a NetworkX graph object
adjG = ToGraph(adj)

# store indices of each cluster in list of lists
model_dict = defaultdict(dict)
all_models = [km, sc] + [km_iso_models[x]['model'] for x in km_iso_models] + [km_pca_models[x]['model'] for x in km_pca_models]
all_names = ['KMeans', 'SpectralClustering'] + list(km_iso_models.keys()) + list(km_pca_models.keys())

# create data frame to store comparison results
model_comparison = pd.DataFrame({'model_name': all_names, 'modularity_score' : [0] * len(all_names), 'conductance_score': [0] * len(all_names)})

for name,model in zip(all_names, all_models):
    
    clusters = []
    
    for i in range(k):
        i_indices = np.where(model.labels_ == i)[0].tolist()
        clusters.append(i_indices)
    model_dict[name]['clusters'] = clusters
    
    Comms = cdlib.NodeClustering(clusters, adjG, 'Manual')
    model_dict[name]['Comms'] = Comms
    
    # Using the object created above, many 'fitness' metrics can be calculated
    modularity = cdlib.evaluation.newman_girvan_modularity(adjG, Comms)
    conductance = cdlib.evaluation.conductance(adjG, Comms)
    model_comparison.loc[model_comparison.model_name == name, ['modularity_score','conductance_score']] = [round(x,4) for x in [modularity.score, conductance.score]]

model_comparison.sort_values(by = ['modularity_score', 'conductance_score'], ascending = [False, True], inplace = True)
model_comparison.reset_index(drop = True, inplace = True)
model_comparison

In [ ]:
# visual check of best performing model
model2viz = model_comparison.iloc[0,0]
selectedDict = km_iso_models if re.search(r'iso', model2viz) is not None else km_pca_models
savepath = puzzle_folder.replace('_','') + '_isoKM_2plot'
img_scatter(selectedDict['isoKM_2']['reduced_features'], pictures, 60, save = False, filename = savepath)

# identify smallest clusters
_, count = np.unique(selectedDict[model2viz]['model'].labels_, return_counts=True)
display_clusters = np.argsort(count)[:3]

# visual inspection of KMeans clusters
cluster = display_clusters[2]
for indx in np.where(selectedDict[model2viz]['model'].labels_ == cluster)[0]:
    save_pic_path = puzzle_folder.replace('_','') + '_cluster' + str(cluster) + '_img' + str(indx)
    display_img(pictures[indx], save = False, filename = save_pic_path)